# Imports

In [1]:
# %load_ext autoreload
# %autoreload 2

In [2]:
# %reload_ext autoreload

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
import time
from itertools import islice
from dataclasses import dataclass
import torchvision
from torchvision.models import densenet161, DenseNet161_Weights, vit_b_16, ViT_B_16_Weights, densenet121, DenseNet121_Weights
import os
import sys
from pathlib import Path
from torchinfo import summary

In [5]:
sys.path.append(str(Path.cwd().parent.parent))
from CheXpert.race_prediction.dataset import CheXpertRaceDataset
from shared_utils import vprint, to_gpu
import shared_utils
from CheXpert.race_prediction.utils import Configs, RaceTrainingMode

# Configs 

In [7]:
@dataclass
class TrainingConfigs(Configs):
    DATA_DIR = os.path.join("..", "..", "data", "CheXpert", "CheXpert-v1.0-small")
    DEMO_FILENAME = "CHEXPERT DEMO.csv"
    TRAIN_LABELS_FILENAME = "train.csv"
    VALID_LABELS_FILENAME = "valid.csv"
    CHECKPOINT_DIR = r"checkpoints"
    DISEASE_PRETRAINED_MODEL_PATH = os.path.join("..", "disease_prediction", "trained_models",
                                                 "2022_07_12-18_47__densenet121_aug__epoch-5__iter-12659__batch_size-16__trainLastLoss-0.3754__validAUC-0.8899.dict")
    BATCH_SIZE = 16
    EPOCHS = 10
    LEARNING_RATE = 1e-4
    LEARNING_RATE_REDUCE_PATIENCE = 3 # number of epochs with no improvement before reducing LR
    LEARNING_RATE_REDUCING_FACTOR = 0.5
    LEARNING_RATE_MIN_VAL = 1e-5
    CHECKPOINT_TIME_INTERVAL = 60*60 # seconds
    MODEL_VERSION = "densenet121_race_denseblock2_shallow"
    FREEZING_POINT = "classifier"
    SHALLOW_DENSEBLCOK = 2
    MODE = RaceTrainingMode.Shallow
    TRAINED_MODEL_PATH = None
    TRAIN_LOADER_SIZE = None
    VALID_LOADER_SIZE = None

In [8]:
shared_utils.set_seed(TrainingConfigs.SEED)

In [10]:
if torch.cuda.is_available():
    vprint(f"Memory info: {torch.cuda.mem_get_info()[0]/10e8:.1f} GB free GPU.", TrainingConfigs)
else: 
    vprint(f"No GPU Memory.", TrainingConfigs)

2022-07-23 20:53: Memory info: 7.6 GB free GPU.


# Training

## Training Setup

In [11]:
train_transform = transforms.Compose([
    transforms.Resize((320,320)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    # augmentation
    transforms.RandomHorizontalFlip(p=0.25),
    transforms.RandomApply([transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.01)], p=0.1),
    transforms.RandomApply([torchvision.transforms.GaussianBlur(kernel_size=(3,3) ,sigma=(0.25, 0.75))], p=0.1),
    torchvision.transforms.RandomAdjustSharpness(sharpness_factor=0.75, p=0.1),
    torchvision.transforms.RandomAdjustSharpness(sharpness_factor=1.25, p=0.1),
])

valid_transform = transforms.Compose([
    transforms.Resize((320,320)),
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [2]:
# Create data loaders.
train_dataset = CheXpertRaceDataset(data_dir=TrainingConfigs.DATA_DIR, demo_filename=TrainingConfigs.DEMO_FILENAME,
                                       labels_filename=TrainingConfigs.TRAIN_LABELS_FILENAME,
                                       transform=train_transform)
train_dataloader = DataLoader(train_dataset, batch_size=TrainingConfigs.BATCH_SIZE, shuffle=False)
TrainingConfigs.TRAIN_LOADER_SIZE = len(train_dataloader)
len(train_dataset)

In [1]:
valid_dataset = CheXpertRaceDataset(data_dir=TrainingConfigs.DATA_DIR, demo_filename=TrainingConfigs.DEMO_FILENAME,
                                       labels_filename=TrainingConfigs.VALID_LABELS_FILENAME,
                                       transform=valid_transform)
valid_dataloader = DataLoader(valid_dataset, batch_size=TrainingConfigs.BATCH_SIZE, shuffle=False)
TrainingConfigs.VALID_LOADER_SIZE = len(valid_dataloader)
len(valid_dataset)

In [14]:
model = densenet121(weights=DenseNet121_Weights.DEFAULT)
num_features = model.classifier.in_features
model.classifier = nn.Sequential(
    nn.Linear(num_features, num_features, bias=True),
    nn.ReLU(),
    nn.Dropout(p=0.1),
    nn.Linear(in_features=num_features, out_features=TrainingConfigs.NUM_DISEASE_CLASSES, bias=True)
)

In [16]:
if TrainingConfigs.MODE == RaceTrainingMode.PartlyFreezed:
    model = shared_utils.load_model(model, TrainingConfigs.DISEASE_PRETRAINED_MODEL_PATH)
if TrainingConfigs.MODE == RaceTrainingMode.Full or TrainingConfigs.MODE == RaceTrainingMode.PartlyFreezed:
    model.classifier[-1] = nn.Linear(in_features=num_features, out_features=TrainingConfigs.NUM_CLASSES, bias=True)

In [18]:
if TrainingConfigs.MODE == RaceTrainingMode.Shallow:
    layer_offset = 3 + 2 * TrainingConfigs.SHALLOW_DENSEBLCOK
    num_features = model.features[layer_offset].norm.num_features
    model = shared_utils.load_portion_of_pretrained_model(model, model.features[:layer_offset])
    classifier_module = nn.Sequential(
        nn.BatchNorm2d(num_features, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
        nn.ReLU(),
        nn.AdaptiveAvgPool2d((1, 1)),   
        nn.Flatten(start_dim=1),
        nn.Linear(in_features=num_features, out_features=num_features, bias=True),
        nn.Dropout(p=0.1),
        nn.Linear(in_features=num_features, out_features=TrainingConfigs.NUM_CLASSES, bias=True)
    )
    model.add_module('classifier', classifier_module)

In [19]:
model

Sequential(
  (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu0): ReLU(inplace=True)
  (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (denseblock1): _DenseBlock(
    (denselayer1): _DenseLayer(
      (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): ReLU(inplace=True)
      (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (denselayer2): _DenseLayer(
      (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (conv1): Conv2d(96, 128, ke

In [20]:
# summary(model, input_size=[16, 3, 320, 320],
#         col_names=("input_size", "output_size", "num_params"))

In [21]:
if TrainingConfigs.MODE == RaceTrainingMode.PartlyFreezed:
    shared_utils.requires_grad_update_by_layer(model, TrainingConfigs, requires_grad=False)

In [22]:
optimizer = torch.optim.Adam(filter(lambda x: x.requires_grad, model.parameters()), lr=TrainingConfigs.LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=TrainingConfigs.LEARNING_RATE_REDUCING_FACTOR,
                                                       patience=TrainingConfigs.LEARNING_RATE_REDUCE_PATIENCE, mode='min',
                                                       min_lr=TrainingConfigs.LEARNING_RATE_MIN_VAL)
criterion = nn.CrossEntropyLoss(reduction='mean')
# requires softmax to retrieve probabilities

## Training Loop 

In [24]:
checkpoint_obj = shared_utils.get_previous_training_place(model, optimizer, scheduler, criterion, TrainingConfigs)
model, optimizer, scheduler, criterion, results, last_epoch, last_iter = checkpoint_obj
score_dict = {
    "auc": "valid_auc",
    "loss": "valid_loss"
}
model.train()
model = to_gpu(model)
start_time = time.time()
shared_utils.start_training_msg(TrainingConfigs)
train_loss_list = []
apply_on_outputs = lambda x: torch.softmax(x, dim=1)
for epoch in range(last_epoch, TrainingConfigs.EPOCHS):
    train_dataloader_iter = tqdm(enumerate(train_dataloader), total=len(train_dataloader))
    if last_iter > -1:
        # fast foward dataloader
        train_dataloader_iter = islice(train_dataloader_iter, last_iter+1, len(train_dataloader))
        last_iter = -1
    for i, (images, labels) in train_dataloader_iter:
        images = to_gpu(images)
        labels = to_gpu(labels)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss_list.append(loss.item())
        if time.time()-start_time > TrainingConfigs.CHECKPOINT_TIME_INTERVAL:
            results['train_loss'].append(sum(train_loss_list)/len(train_loss_list))
            train_loss_list = []
            shared_utils.create_checkpoint(model, optimizer, scheduler, criterion, epoch, i, valid_dataloader,
                                           results, TrainingConfigs, score_dict, 
                                           apply_on_outputs=apply_on_outputs,
                                           by_study=None, challenge_ann_only=None)
            assert model.training
            start_time = time.time()
    shared_utils.create_checkpoint(model, optimizer, scheduler, criterion, epoch, len(train_dataloader), valid_dataloader, 
                                   results, TrainingConfigs, score_dict, apply_on_outputs=apply_on_outputs,
                                   by_study=None, challenge_ann_only=None)
    scheduler.step(results["valid_loss"][-1])

2022-07-23 20:54: Loading model - checkpoints/2022_07_21-13_19__densenet121_race_denseblock2_shallow__epoch-9__iter-9968__batch_size-16__trainLastLoss-0.3791__validAUC-0.8807.dict
2022-07-23 20:54: 
2022-07-23 20:54: ----------------------------------------------------------------------------------------------------
2022-07-23 20:54: ----------------------------------------------------------------------------------------------------
2022-07-23 20:54: 
2022-07-23 20:54: Start Training


  0%|          | 0/10266 [00:00<?, ?it/s]

KeyboardInterrupt: 